Google Colab上執行  
使用Lena，放在google colab的sample_data目錄下後再進行存取  
SourceCode@Google Colab: https://colab.research.google.com/drive/1y2QkKbeeFNnLZBwtRVOOVQOc0KmJMVGz

In [0]:
# _*_ coding: utf-8 _*_
# _*_ coding: cp950 _*_

# author: Chi-Hsu Chen (css920@gmail.com)

import numpy as np
import cv2
import time
from google.colab.patches import cv2_imshow

from keras import utils
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Model,Sequential
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D
from keras.layers import Input,GlobalAveragePooling2D,GlobalMaxPooling2D


VGG16模型架構
![VGG16模型](https://github.com/ChiHsuChen/DL-CVMarathon/blob/master/homework/VGG16_p.png?raw=true)

# VGG16模型建構

In [0]:
def VGG16(include_top=True,pooling='max'):
  # include_top=True -> 加入全連接層作為分類使用
  # include_top=False -> 只用VGG16作為擷取特徵

  model=Sequential()
  
  # 輸入大小: 224*224*1
  input_shape=(224,224,3)
  
  # block 1
  model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same',activation='relu',input_shape=input_shape,name='block1_conv1'))
  model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same',activation='relu',name='block1_conv2'))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name='block1_maxpooling1'))
  
  # block 2
  model.add(Conv2D(filters=128,kernel_size=(3,3),padding='same',activation='relu',name='block2_conv1'))
  model.add(Conv2D(filters=128,kernel_size=(3,3),padding='same',activation='relu',name='block2_conv2'))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name='block2_maxpooling1'))
  
  # block 3
  model.add(Conv2D(filters=256,kernel_size=(3,3),padding='same',activation='relu',name='block3_conv1'))
  model.add(Conv2D(filters=256,kernel_size=(3,3),padding='same',activation='relu',name='block3_conv2'))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name='block3_maxpooling1'))
  
  # block 4
  model.add(Conv2D(filters=512,kernel_size=(3,3),padding='same',activation='relu',name='block4_conv1'))
  model.add(Conv2D(filters=512,kernel_size=(3,3),padding='same',activation='relu',name='block4_conv2'))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name='block4_maxpooling1'))
  
  # block 5
  model.add(Conv2D(filters=512,kernel_size=(3,3),padding='same',activation='relu',name='block5_conv1'))
  model.add(Conv2D(filters=512,kernel_size=(3,3),padding='same',activation='relu',name='block5_conv2'))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name='block5_maxpooling1'))
  
  if include_top==True:
    # block6
    # Flatten
    model.add(Flatten(name='block6_flatten'))
    # FC 1 - 4096
    model.add(Dense(4096,activation='relu',name='block6_fc1'))
    # FC 2 - 4096
    model.add(Dense(4096,activation='relu',name='block6_fc2'))
    # FC 3 - 1000
    model.add(Dense(1000,activation='softmax',name='block6_output'))
  else:
    if pooling=='max':
      model.add(GlobalMaxPooling2D())
    elif pooling=='avg':
      model.add(GlobalAveragePooling2D())

  return model



In [29]:
# 包含全連接層
vgg16_model_INCLUDE=VGG16(include_top=True)
vgg16_model_INCLUDE.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_maxpooling1 (MaxPooli (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_maxpooling1 (MaxPooli (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)     

In [28]:
# 不包含全連接層
vgg16_model_NOTINCLUDE=VGG16(include_top=False)
vgg16_model_NOTINCLUDE.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_maxpooling1 (MaxPooli (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_maxpooling1 (MaxPooli (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)     